## Метод штрафов

In [28]:
import numpy as np
import sympy as sp
import pandas as pd

rs = [1,2,10,100,1000,10**50]
e = 1e-3

#### пример
# f = lambda x: x[0]**2 + x[1]**2
# g = lambda x: x[0] + x[1] - 2
# P = lambda x, r: (r/2) * g(x)**2

# x1 = lambda r: r/(1+r)
# x2 = lambda r: r/(1+r)
#####


f = lambda x: x[0]**2 + 5*x[1]**2 - x[0]*x[1] + x[0]
g = lambda x: x[0] + x[1] - 1
P = lambda x, r: (r/2) * g(x)**2

x2 = lambda r: (4*r-1)/(19+14*r)
x1 = lambda r: (11*x2(r)-1)/3

def hess():
    x1, x2, r = sp.symbols('x1 x2 r')
    F = x1 ** 2 + 5 * x2 ** 2 - x1*x2 + x1 + (r/2) * (x1 + x2 - 1)

    f1 = sp.diff(F, x1)
    f2 = sp.diff(F, x2)

    f11 = sp.diff(f1, x1)
    f12 = sp.diff(f1, x2)
    f22 = sp.diff(f2, x2)
    
    hessian = sp.Matrix([[f11, f12], [f12, f22]])
    minor1 = hessian[0, 0]
    minor2 = hessian.det()

    return minor1 > 0 and minor2 > 0

def newton_method(r, max_iter=100):
    x = np.array([0.5, 0.5])
    
    def equations(x):
        x1, x2 = x
        eq1 = 2*x1 - x2 + 1 + r * (x1 + x2 - 1)
        eq2 = 10*x2 - x1 + r * (x1 + x2 - 1)
        return np.array([eq1, eq2])
    
    
    for _ in range(max_iter):
        F = equations(x)
        H = np.array([[2 + r, r], [r, 10 + r]])
        delta_x = np.linalg.solve(H, -F)
        x = x + delta_x
        
        if np.linalg.norm(delta_x) < e:
            break
    
    return x

def penalty_method():
    data = {'r':[], 'x1':[], 'x2':[], 'F':[], 'P':[]}
    x = [0, 0]
    for r in rs:
        data['r'].append(r)
        # x[0] = x1(r)
        # x[1] = x2(r)
        x = newton_method(r)
        data['x1'].append(x[0])
        data['x2'].append(x[1])
        Pk = P(x, r)
        F = f(x) + Pk
        data['F'].append(F)
        data['P'].append(Pk)
        if Pk <= e:
            break
    return pd.DataFrame(data=data)


print(hess())
penalty_method()


True


UFuncTypeError: Cannot cast ufunc 'solve1' input 0 from dtype('O') to dtype('float64') with casting rule 'same_kind'

## Метод множителей

In [33]:
import pandas as pd

rs = [1,2,10,100,1000]
e = 1e-3

f = lambda x: x[0]**2 + 5*x[1]**2 - x[0]*x[1] + x[0]
g = lambda x: x[0] + x[1] - 1
l = lambda x, r, l: l + r*g(x)
P = lambda x, r, l: ((r/2) * g(x) + l) * g(x)


x2 = lambda r, l: (4*r-1-3*l)/(19+14*r)
x1 = lambda r, l: (11*x2(r,l)-1)/3

def hess():
    x1, x2, r = sp.symbols('x1 x2 r l')
    F = x1 ** 2 + 5 * x2 ** 2 - x1*x2 + x1 + (r/2) * (x1 + x2 - 1)

    f1 = sp.diff(F, x1)
    f2 = sp.diff(F, x2)

    f11 = sp.diff(f1, x1)
    f12 = sp.diff(f1, x2)
    f22 = sp.diff(f2, x2)
    
    hessian = sp.Matrix([[f11, f12], [f12, f22]])
    minor1 = hessian[0, 0]
    minor2 = hessian.det()

    return minor1 > 0 and minor2 > 0

def newton_method(r, l, max_iter=100):
    x = np.array([0.5, 0.5])
    
    def equations(x):
        x1, x2 = x
        eq1 = 2*x1 - x2 + 1 + r * (x1 + x2 - 1) + l
        eq2 = 10*x2 - x1 + r * (x1 + x2 - 1) + l
        return np.array([eq1, eq2])
    
    
    for _ in range(max_iter):
        F = equations(x)
        H = np.array([[2 + r, r], [r, 10 + r]])
        delta_x = np.linalg.solve(H, -F)
        x = x + delta_x
        
        if np.linalg.norm(delta_x) < e:
            break
    
    return x


def penalty_method():
    data = {'r':[], 'l':[], 'x1':[], 'x2':[], 'P':[]}
    x = [0, 0]
    lk = 0
    for r in rs:
        data['r'].append(r)
        data['l'].append(lk)
        x[0] = x1(r, lk)
        x[1] = x2(r, lk)
        # x = newton_method(r, lk)
        data['x1'].append(x[0])
        data['x2'].append(x[1])
        Pk = P(x, r, lk)
        data['P'].append(Pk)
        if Pk <= e:
            break
        lk = l(x, r, lk)
    print(f(x))
    return pd.DataFrame(data=data)


penalty_method()



1.4285697208366832


,r,l,x1,x2,P
0,1,0.000000,0.000000,0.090909,0.413223
1,2,-0.909091,0.425532,0.206963,0.469155
2,10,-1.644101,0.679781,0.276304,0.081845
3,100,-2.083257,0.713824,0.285588,0.001242
4,1000,-2.142059,0.714285,0.285714,0.000002
